In [22]:
import os
import json
import numpy as np
import pandas as pd
from pprint import pprint

from openai import OpenAI

from IPython.display import display

In [5]:
PATH_CONFIG = '../config/'
PATH_DATA = '../data/'

CONFIG = 'openai.json'
CCS = 'ccs.csv'
CLAIM = 'claim_lines.csv'
PRESCRIPTION = 'prescription_drugs.csv'
DETAILED_CLAIM = 'claim_detail.csv'

Step 1: 

Create a csv by combining claim_lins.csv and ccs.csv


In [6]:
df_ccs = pd.read_csv(os.path.join(PATH_DATA, CCS))
df_claim = pd.read_csv(os.path.join(PATH_DATA, CLAIM))

In [12]:
df_ccs.head()

,diag,diag_desc,ccs_1_desc,ccs_2_desc,ccs_3_desc
0,A000,"Cholera due to Vibrio cholerae 01, biovar chol...",Diseases of the digestive system,Intestinal infection [135.],Intestinal infection
1,A001,"Cholera due to Vibrio cholerae 01, biovar eltor",Diseases of the digestive system,Intestinal infection [135.],Intestinal infection
2,A009,"Cholera, unspecified",Diseases of the digestive system,Intestinal infection [135.],Intestinal infection
3,A0100,"Typhoid fever, unspecified",Diseases of the digestive system,Intestinal infection [135.],Intestinal infection
4,A0101,Typhoid meningitis,Diseases of the nervous system and sense organs,Central nervous system infection,Meningitis (except that caused by tuberculosis...


In [13]:
df_claim.head()

,record_id,member_id,date_svc,diag1
0,57738,M0000001,2015-12-06,N92.6
1,57750,M0000001,2015-12-06,O26.842
2,65072,M0000001,2015-12-13,O26.842
3,201796,M0000001,2016-02-29,O26.843
4,267197,M0000001,2016-03-27,O26.843


In [7]:
df_claim['diag'] = df_claim['diag1'].str.replace('.', '')

/var/folders/p7/37gvwfj126vbz0f_w_nr2c880000gn/T/ipykernel_53732/3697870285.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_claim['diag'] = df_claim['diag1'].str.replace('.', '')


In [8]:
df_claim_detail = pd.merge(df_claim, df_ccs, on = 'diag', how = 'left')
df_claim_detail = df_claim_detail.drop(['record_id', 'diag1'], axis = 1)
df_claim_detail.head()

,member_id,date_svc,diag,diag_desc,ccs_1_desc,ccs_2_desc,ccs_3_desc
0,M0000001,2015-12-06,N926,"Irregular menstruation, unspecified",Diseases of the genitourinary system,Diseases of female genital organs,Menstrual disorders
1,M0000001,2015-12-06,O26842,"Uterine size-date discrepancy, second trimester",Complications of pregnancy; childbirth; and th...,Complications mainly related to pregnancy,Other complications of pregnancy
2,M0000001,2015-12-13,O26842,"Uterine size-date discrepancy, second trimester",Complications of pregnancy; childbirth; and th...,Complications mainly related to pregnancy,Other complications of pregnancy
3,M0000001,2016-02-29,O26843,"Uterine size-date discrepancy, third trimester",Complications of pregnancy; childbirth; and th...,Complications mainly related to pregnancy,Other complications of pregnancy
4,M0000001,2016-03-27,O26843,"Uterine size-date discrepancy, third trimester",Complications of pregnancy; childbirth; and th...,Complications mainly related to pregnancy,Other complications of pregnancy


In [9]:
df_claim_detail.to_csv(os.path.join(PATH_DATA, DETAILED_CLAIM))

In [100]:
def report(member_id, df):
    member_df = df[df['member_id'] == member_id]
    
    display(member_df)
    
    start_date = member_df['date_svc'].min()
    end_date = member_df['date_svc'].max()
    
    diags = member_df.groupby('diag')
    
    num_diag = diags.ngroups
    if (num_diag == 1): diagnosis_analysis = f'only one'
    elif (num_diag > 1 and num_diag <= 5): diagnosis_analysis = f'few (to be more specific {num_diag}) different'
    elif (num_diag > 5 and num_diag <= 10): diagnosis_analysis = f'some (to be more specific {num_diag}) different'
    else: diagnosis_analysis = f'several (to be more specific {num_diag}) different'
    
    history = None
    if (member_df.shape[0] != num_diag):
        diagnosis_counts = member_df.groupby('ccs_3_desc').size()
        # Sorting counts in descending order, and if counts are equal, sorting by date of diagnosis
        top_diagnoses = diagnosis_counts.reset_index(name = 'Count').sort_values(by = ['Count', 'ccs_3_desc'], ascending=[False, True])
        
        if (num_diag == 1): 
            history = f"Their biggest and only concern is {top_diagnoses.head(1)['ccs_3_desc'].values[0]}."
        elif (num_diag > 1 and num_diag <= 5):
            history = f"Amongst others, their biggest concern is {top_diagnoses.head(1)['ccs_3_desc'].values[0]}."
        elif (num_diag > 5 and num_diag <= 10):
            top_2_diag = top_diagnoses.head(2)
            diag_1 = top_2_diag['ccs_3_desc'].values[0]
            diag_2 = top_2_diag['ccs_3_desc'].values[1]
            history = f"Amongst others, their biggest concerns are {diag_1} and {diag_2}."
        else:
            top_4_diag = top_diagnoses.head(4)
            diag_1 = top_4_diag['ccs_3_desc'].values[0]
            diag_2 = top_4_diag['ccs_3_desc'].values[1]
            diag_3 = top_4_diag['ccs_3_desc'].values[2]
            diag_4 = top_4_diag['ccs_3_desc'].values[3]
            history = f"Amongst others, their biggest concerns are {diag_1}, {diag_2}, {diag_3} and {diag_4}."
    
    print(f'Member {member_id} has been coming in since {start_date}. Their latest visit was {end_date}.')
    print(f'They had {diagnosis_analysis} diagnosis during the time period.')
    if history: print(history)
    print(f"Their latest concern is {member_df['diag_desc'].iloc[-1]}")


In [101]:
member_id = df_claim_detail['member_id'].sample(n = 1).item()
# report(member_id, df_claim_detail)
# report('M0000264', df_claim_detail)
report('M0226366', df_claim_detail)


,member_id,date_svc,diag,diag_desc,ccs_1_desc,ccs_2_desc,ccs_3_desc
1487702,M0226366,2016-05-23,J028,Acute pharyngitis due to other specified organ...,Diseases of the respiratory system,Respiratory infections,Other upper respiratory infections


Member M0226366 has been coming in since 2016-05-23. Their latest visit was 2016-05-23.
They had only one diagnosis during the time period.
Their latest concern is Acute pharyngitis due to other specified organisms


In [35]:
SYSTEM_ROLE = 'Youre a helpful bot that builds a characterization of a member’s health status based on their outpatient data in 3 sentences'

def get_openai_message(history):
    message = []
    message.append({'role': 'system', 'content': SYSTEM_ROLE})
    message.append({'role': 'user', 'content': 'Patient history: {}'.format(history)})
    return message

def report_llm(member_id, df):
    with open(os.path.join(PATH_CONFIG, CONFIG)) as file:
        info = json.load(file)
    openai = OpenAI(
        api_key = info['openai']['key']
    )
    member_df = df[df['member_id'] == member_id]

    start_date = member_df['date_svc'].min()
    end_date = member_df['date_svc'].max()
    diags = member_df.groupby('diag')
    num_diag = diags.ngroups
    if (num_diag == 1): diagnosis_analysis = f'only one'
    elif (num_diag > 1 and num_diag <= 5): diagnosis_analysis = f'few (to be more specific {num_diag}) different'
    elif (num_diag > 5 and num_diag <= 10): diagnosis_analysis = f'some (to be more specific {num_diag}) different'
    else: diagnosis_analysis = f'several (to be more specific {num_diag}) different'

    member_df['diagnosis'] = member_df['diag_desc'] + ' ' + \
                             member_df['ccs_1_desc'] + ' ' + \
                             member_df['ccs_2_desc'] + ' ' + \
                             member_df['ccs_3_desc']
    
    member_df = member_df[['date_svc', 'diagnosis']]
    display(member_df)
    history = member_df.to_dict('records')

    response = openai.chat.completions.create(
        model = 'gpt-3.5-turbo',
        messages = get_openai_message(history),
    )

    
    print(f'Member {member_id} has been coming in since {start_date}. Their latest visit was {end_date}.')
    print(f'They had {diagnosis_analysis} diagnosis during the time period.')
    print(response.choices[0].message.content)
    

In [36]:
member_id = df_claim_detail['member_id'].sample(n = 1).item()
# report_llm(member_id, df_claim_detail)
report_llm('M0091717', df_claim_detail)

/var/folders/p7/37gvwfj126vbz0f_w_nr2c880000gn/T/ipykernel_53398/1347410672.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  member_df['diagnosis'] = member_df['diag_desc'] + ' ' + \


,date_svc,diagnosis
607218,2016-02-27,Frequency of micturition Diseases of the genit...
607219,2016-04-16,"Acute upper respiratory infection, unspecified..."
607220,2016-04-23,Encntr for routine child health exam w/o abnor...
607221,2016-06-25,"Constipation, unspecified Diseases of the dige..."
607222,2016-07-01,"Acute pharyngitis, unspecified Diseases of the..."
607223,2016-10-29,"Urinary tract infection, site not specified Di..."
607224,2016-12-03,Encounter for immunization Infectious and para...
607225,2017-01-20,Acute obstructive laryngitis [croup] Diseases ...


Member M0091717 has been coming in since 2016-02-27. Their latest visit was 2017-01-20.
They had some (to be more specific 8) different diagnosis during the time period.
Based on the outpatient data, the patient has a history of genitourinary issues including urinary frequency and urinary tract infections, as well as respiratory problems such as acute upper respiratory infections and pharyngitis. Additionally, there are instances of constipation and routine health exams in the record, suggesting a varied but relatively common set of health concerns. Regular immunizations are also noted in the history, indicating proactive healthcare behavior to prevent infectious diseases.
